In [1]:
import torch

print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
12.4
NVIDIA GeForce RTX 4070


In [3]:
import os
import logging
import argparse
import ast
from typing import Tuple, Dict, Any

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import torch

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from datasets import Dataset

# Настройка логирования
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

In [ ]:
def load_and_preprocess_data(
    json_file: str, rare_threshold: int = 50
) -> Tuple[pd.DataFrame, Dict[str, int], Dict[int, str]]:
    """
    Загружает данные из JSON-файла, выполняет парсинг тегов, группирует схожие классы
    в высокоуровневые категории, а также фильтрует редкие классы.

    Аргументы:
        json_file (str): путь к JSON-файлу с данными.
        rare_threshold (int): минимальное количество примеров для того, чтобы класс не считался редким.

    Возвращает:
        Tuple[pd.DataFrame, Dict[str, int], Dict[int, str]]:
            - DataFrame с колонками 'title', 'summary', 'tag', 'parsed_tag', 'mapped_tag', 'text', 'label'
            - Словарь tag2id: отображает высокоуровневую категорию в числовой идентификатор
            - Словарь id2tag: отображает числовой идентификатор в метку
    """
    df: pd.DataFrame = pd.read_json(json_file)
    logger.info(f"Загружено {len(df)} примеров")

    # Удаляем записи без необходимых полей
    df = df.dropna(subset=["title", "summary", "tag"]).reset_index(drop=True)

    # Парсим исходный тег (например, "cs.AI")
    def parse_first_tag(tag_str: str) -> Any:
        try:
            tags = ast.literal_eval(tag_str)
            if tags and isinstance(tags, list):
                return tags[0].get("term", None)
            return None
        except Exception as e:
            logger.error(f"Ошибка при разборе тега: {e}")
            return None

    df["parsed_tag"] = df["tag"].apply(parse_first_tag)
    df = df.dropna(subset=["parsed_tag"]).reset_index(drop=True)

    # Группируем тег в высокоуровневую категорию
    df["mapped_tag"] = df["parsed_tag"].apply(map_tag)

    # Объединяем title и summary для формирования входного текста
    df["text"] = df["title"] + " [SEP] " + df["summary"]

    # Смотрим распределение новых меток
    tag_counts = df["mapped_tag"].value_counts()
    logger.info("Распределение классов после группировки:")
    logger.info(f"\n{tag_counts}")

    # Фильтрация: редкие классы (менее rare_threshold примеров) объединяются в общий класс "Other"
    rare_tags = tag_counts[tag_counts < rare_threshold].index.tolist()
    if rare_tags:
        logger.info(
            f"Классы, встречающиеся менее {rare_threshold} раз и будут заменены на 'Other': {rare_tags}"
        )
        df.loc[df["mapped_tag"].isin(rare_tags), "mapped_tag"] = "Other"
        # Пересчитаем распределение
        tag_counts = df["mapped_tag"].value_counts()
        logger.info("Распределение классов после объединения редких классов в 'Other':")
        logger.info(f"\n{tag_counts}")

    # Создаем отображения меток в числовые id
    unique_tags = sorted(df["mapped_tag"].unique())
    tag2id: Dict[str, int] = {tag: idx for idx, tag in enumerate(unique_tags)}
    id2tag: Dict[int, str] = {idx: tag for tag, idx in tag2id.items()}

    df["label"] = df["mapped_tag"].map(tag2id)

    logger.info(f"Будет использоваться {len(unique_tags)} классов: {unique_tags}")
    return df, tag2id, id2tag

In [7]:
data, _, _ = load_and_preprocess_data("/home/anufriev/Projects/cis/data/arxivData.json")
data

2025-04-08 03:54:57,580 - INFO - __main__ - Загружено 41000 примеров
2025-04-08 03:54:58,147 - INFO - __main__ - Найдено 126 уникальных меток


,author,day,id,link,month,summary,tag,title,year,parsed_tag,text,label
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018,cs.AI,Dual Recurrent Attention Units for Visual Ques...,15
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016,cs.CL,Sequential Short-Text Classification with Recu...,20
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016,cs.CL,Multiresolution Recurrent Neural Networks: An ...,20
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017,stat.ML,Learning what to share between loosely related...,124
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017,cs.CL,A Deep Reinforcement Learning Chatbot [SEP] We...,20
...,...,...,...,...,...,...,...,...,...,...,...,...
40995,"[{'name': 'Vitaly Feldman'}, {'name': 'Pravesh...",18,1404.4702v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We study the complexity of learning and approx...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Nearly Tight Bounds on $\ell_1$ Approximation ...,2014,cs.LG,Nearly Tight Bounds on $\ell_1$ Approximation ...,37
40996,"[{'name': 'Orly Avner'}, {'name': 'Shie Mannor'}]",22,1404.5421v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We consider the problem of multiple users targ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Concurrent bandits and cognitive radio networks,2014,cs.LG,Concurrent bandits and cognitive radio network...,37
40997,"[{'name': 'Ran Zhao'}, {'name': 'Deanna Needel...",22,1404.5899v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,"In this paper, we compare and analyze clusteri...","[{'term': 'math.NA', 'scheme': 'http://arxiv.o...",A Comparison of Clustering and Missing Data Me...,2014,math.NA,A Comparison of Clustering and Missing Data Me...,80
40998,"[{'name': 'Zongyan Huang'}, {'name': 'Matthew ...",25,1404.6369v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",4,Cylindrical algebraic decomposition(CAD) is a ...,"[{'term': 'cs.SC', 'scheme': 'http://arxiv.org...",Applying machine learning to the problem of ch...,2014,cs.SC,Applying machine learning to the problem of ch...,50


In [13]:
data["parsed_tag"].value_counts().head(50)

parsed_tag
cs.CV              11580
cs.LG               6355
cs.AI               6027
cs.CL               4930
stat.ML             4474
cs.NE               1809
cs.IR                543
cs.RO                433
math.OC              353
cs.LO                253
cs.SI                221
cs.DS                185
cs.SD                178
cs.CR                178
stat.ME              169
q-bio.NC             169
cs.DB                157
cs.GT                156
cs.IT                151
cs.HC                148
cs.DC                146
cs.CY                128
cmp-lg               110
cs.CE                107
cs.SE                104
cs.MM                103
cs.MA                 95
math.ST               94
q-bio.QM              88
cs.NI                 86
physics.soc-ph        80
stat.AP               76
cs.NA                 72
cs.SY                 68
quant-ph              66
stat.CO               59
cs.PL                 55
cs.CC                 55
cs.GR                 53
cs.ET         